# Unión de datos scrapeados, Dataframe final 

In [1]:
!ls -lrt

total 33728
-rw-r--r--  1 prealfer  staff    420474 31 ago 19:15 TFM_scraping.ipynb
-rw-r--r--  1 prealfer  staff         0  5 sep 23:51 README.md
-rw-r--r--  1 prealfer  staff         0  5 sep 23:51 README.txt
-rw-r--r--  1 prealfer  staff    184491  6 sep 23:02 car_models.csv
-rw-r--r--  1 prealfer  staff   2352266 25 sep 23:32 listado_modelos_3.txt
-rw-r--r--  1 prealfer  staff     87691 28 sep 00:34 precios_versiones
-rw-r--r--  1 prealfer  staff   1584860 30 sep 18:04 versiones_con_precio
-rw-r--r--  1 prealfer  staff     15285 30 sep 20:02 maleteros
-rw-r--r--  1 prealfer  staff     16680 30 sep 20:06 Scrapeo-Maleteros.ipynb
-rw-r--r--  1 prealfer  staff    500156 30 sep 20:07 Script scrapping - Dataframe.ipynb
-rw-r--r--  1 prealfer  staff  12086763  1 oct 17:29 Construcción Dataframe definitivo.ipynb


In [1]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
datos = pd.read_csv('Dataframe_limpio.csv',header=0)
datos = datos.drop('Unnamed: 0',axis=1)
datos.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso
0,Abarth,124 Spider,Abarth 1.4 170,35.093€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg
1,Abarth,124 Spider,Abarth 1.4 Scorpione 170,36.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg
2,Abarth,124 Spider,Abarth 1.4 Aut. 170,37.961€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg
3,Abarth,124 Spider,Abarth 1.4 Scorpione Aut. 170,38.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg
4,Abarth,595,Abarth 1.4T,Jet 145,15.670€,,Gasolina,Tracción delantera,145 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,205 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,3


In [5]:
maleteros = pd.read_csv('maleteros')
maleteros = maleteros.drop('Unnamed: 0',axis=1)
maleteros.head()

,Marca,Modelo,CapacidadMaletero
0,ALFA ROMEO,Alfa Romeo MiTo,270
1,ALFA ROMEO,Alfa Romeo 4C,110
2,ALFA ROMEO,Alfa Romeo Giulietta,350
3,ALFA ROMEO,Alfa Romeo Giulia,480
4,ALFA ROMEO,Nuevo Alfa Romeo Stelvio 2017,525


In [6]:
datos['marca'] = datos['marca'].apply(lambda x: x.upper().strip())
datos['modelo'] = datos['modelo'].apply(lambda x: x.upper().strip())
maleteros['Modelo'] = maleteros['Modelo'].apply(lambda x: x.upper())

In [7]:
datos.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso
0,ABARTH,124 SPIDER,Abarth 1.4 170,35.093€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37.961€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg
4,ABARTH,595,Abarth 1.4T,Jet 145,15.670€,,Gasolina,Tracción delantera,145 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,205 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,3


In [8]:
maleteros.head()

,Marca,Modelo,CapacidadMaletero
0,ALFA ROMEO,ALFA ROMEO MITO,270
1,ALFA ROMEO,ALFA ROMEO 4C,110
2,ALFA ROMEO,ALFA ROMEO GIULIETTA,350
3,ALFA ROMEO,ALFA ROMEO GIULIA,480
4,ALFA ROMEO,NUEVO ALFA ROMEO STELVIO 2017,525


In [9]:
datos['marca-modelo']=datos['marca']+' '+datos['modelo']

In [10]:
datos.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso,marca-modelo
0,ABARTH,124 SPIDER,Abarth 1.4 170,35.093€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg,ABARTH 124 SPIDER
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg,ABARTH 124 SPIDER
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37.961€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg,ABARTH 124 SPIDER
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg,ABARTH 124 SPIDER
4,ABARTH,595,Abarth 1.4T,Jet 145,15.670€,,Gasolina,Tracción delantera,145 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,205 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,3,ABARTH 595


In [11]:
maleteros.columns = ['Marca','marca-modelo','Maletero']

In [12]:
maleteros.head()

,Marca,marca-modelo,Maletero
0,ALFA ROMEO,ALFA ROMEO MITO,270
1,ALFA ROMEO,ALFA ROMEO 4C,110
2,ALFA ROMEO,ALFA ROMEO GIULIETTA,350
3,ALFA ROMEO,ALFA ROMEO GIULIA,480
4,ALFA ROMEO,NUEVO ALFA ROMEO STELVIO 2017,525


In [13]:
def busca_capacidad(modelo):
    return maleteros[maleteros['marca-modelo']==modelo].Maletero[1]
    
    
    

In [33]:
resultado = pd.concat([datos, maleteros], axis=1, join='outer')

In [26]:
resultado = datos.merge(maleteros,how='outer',on='marca-modelo')

In [34]:
resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5927 entries, 0 to 5926
Data columns (total 37 columns):
marca                  5927 non-null object
modelo                 5927 non-null object
version                5927 non-null object
precio                 5927 non-null object
puertas                5927 non-null object
combustible            5927 non-null object
traccion               5927 non-null object
cv                     5927 non-null object
cambio                 5927 non-null object
n_marchas              5927 non-null object
co2                    5927 non-null object
autonomia              5927 non-null object
consumo                5927 non-null object
airbags                5927 non-null object
garantia               5927 non-null object
cilindrada             5927 non-null object
capacidad deposito     5927 non-null object
marchas                5927 non-null object
max_vel                5927 non-null object
0_a_100                5927 non-null object
rpm          

In [35]:
resultado.head(100)

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso,marca-modelo,Marca,marca-modelo,Maletero
0,ABARTH,124 SPIDER,Abarth 1.4 170,35.093€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg,ABARTH 124 SPIDER,ALFA ROMEO,ALFA ROMEO MITO,270.0
1,ABARTH,124 SPIDER,Abarth 1.4 Scorpione 170,36.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg,ABARTH 124 SPIDER,ALFA ROMEO,ALFA ROMEO 4C,110.0
2,ABARTH,124 SPIDER,Abarth 1.4 Aut. 170,37.961€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg,ABARTH 124 SPIDER,ALFA ROMEO,ALFA ROMEO GIULIETTA,350.0
3,ABARTH,124 SPIDER,Abarth 1.4 Scorpione Aut. 170,38.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg,ABARTH 124 SPIDER,ALFA ROMEO,ALFA ROMEO GIULIA,480.0
4,ABARTH,595,Abarth 1.4T,Jet 145,15.670€,,Gasolina,Tracción delantera,145 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,205 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,3,ABARTH 595,ALFA ROMEO,NUEVO ALFA ROMEO STELVIO 2017,525.0
5,ABARTH,595,Abarth C 1.4T,Jet 145,17.316€,,Gasolina,Tracción delantera,145 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,205 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,2,ABARTH 595,ASTON_MARTIN,ASTON MARTIN V12 VANTAGE S,300.0
6,ABARTH,595,Abarth 1.4T,Jet Turismo 165,20.497€,,Gasolina,Tracción delantera,165 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,210 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,3,ABARTH 595,ASTON_MARTIN,NUEVO ASTON MARTIN DB11 2017,270.0
7,ABARTH,595,Abarth 1.4T,Jet 145 Aut.,21.086€,,Gasolina,Tracción delantera,145 cv,Automática secuencial,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,205 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,3,ABARTH 595,ASTON_MARTIN,ASTON MARTIN RAPIDE S,317.0
8,ABARTH,595,Abarth 1.4T,Jet Pista 160,21.861€,,Gasolina,Tracción delantera,160 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,210 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,3,ABARTH 595,ASTON_MARTIN,ASTON MARTIN VANQUISH COUPE,368.0
9,ABARTH,595,Abarth C 1.4T,Jet Turismo 165,22.208€,,Gasolina,Tracción delantera,165 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,210 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,2,ABARTH 595,ASTON_MARTIN,ASTON MARTIN VANQUISH VOLANTE,279.0


In [36]:
resultado.tail(50)

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso,marca-modelo,Marca,marca-modelo,Maletero
5877,VOLVO,V90,D5 R,Design AWD Aut.,59.209€,,Diesel,Tracción total permanente,235 cv,Automática secuencial,8 marchas,129 gr/km,"1.122,4 km","4,9 l/100km",0,129 gr/km,Tracción total permanente,"1.122,4 km",Tracción total permanente,230 km/h,"4,9 l/100km",4000 rpm,0,Sí,No,,No,No,No,No,NaN,No,5,VOLVO V90,NaN,NaN,NaN
5878,VOLVO,V90,D5 Inscription AWD Aut.,60.582€,5 puertas,Diesel,Tracción total permanente,235 cv,Automática secuencial,8 marchas,129 gr/km,"1.122,4 km","4,9 l/100km",0,24 meses,1.969 cc,55 lts.,8,230 km/h,"8,20 s.",4000 rpm,480Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Stationwagon,1791 kg,VOLVO V90,NaN,NaN,NaN
5879,VOLVO,V90,T6 Momentum AWD 320 Aut.,64.516€,5 puertas,Gasolina,Tracción total permanente,320 cv,Automática secuencial,8 marchas,169 gr/km,"743,2 km","7,4 l/100km",0,24 meses,1.969 cc,55 lts.,8,250 km/h,"5,90 s.",5700 rpm,400Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Stationwagon,1791 kg,VOLVO V90,NaN,NaN,NaN
5880,VOLVO,V90,T6 R,Design AWD 320 Aut.,66.836€,,Gasolina,Tracción total permanente,320 cv,Automática secuencial,8 marchas,169 gr/km,"743,2 km","7,4 l/100km",0,169 gr/km,Tracción total permanente,"743,2 km",Tracción total permanente,250 km/h,"7,4 l/100km",5700 rpm,0,Sí,No,,No,No,No,No,NaN,No,5,VOLVO V90,NaN,NaN,NaN
5881,VOLVO,V90,T6 Inscription AWD 320 Aut.,68.264€,5 puertas,Gasolina,Tracción total permanente,320 cv,Automática secuencial,8 marchas,169 gr/km,"743,2 km","7,4 l/100km",0,24 meses,1.969 cc,55 lts.,8,250 km/h,"5,90 s.",5700 rpm,400Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Stationwagon,1791 kg,VOLVO V90,NaN,NaN,NaN
5882,VOLVO,V90,T8 Momentum Twin AWD,71.050€,4 puertas,Gasolina y corriente eléctrica,Tracción total permanente,407 cv,Automática secuencial,8 marchas,46 gr/km,"2.500,0 km","2,0 l/100km",0,24 meses,1.969 cc,50 lts.,8,250 km/h,"4,70 s.",0 rpm,0Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Berlina,2077 kg,VOLVO V90,NaN,NaN,NaN
5883,VOLVO,V90,T8 R,Design Twin AWD,72.702€,,Gasolina y corriente eléctrica,Tracción total permanente,407 cv,Automática secuencial,8 marchas,46 gr/km,"2.500,0 km","2,0 l/100km",0,46 gr/km,Tracción total permanente,"2.500,0 km",Tracción total permanente,250 km/h,"2,0 l/100km",0 rpm,0,Sí,No,,No,No,No,No,NaN,No,4,VOLVO V90,NaN,NaN,NaN
5884,VOLVO,V90,T8 Inscription Twin AWD,74.353€,4 puertas,Gasolina y corriente eléctrica,Tracción total permanente,407 cv,Automática secuencial,8 marchas,46 gr/km,"2.500,0 km","2,0 l/100km",0,24 meses,1.969 cc,50 lts.,8,250 km/h,"4,70 s.",0 rpm,0Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Berlina,2077 kg,VOLVO V90,NaN,NaN,NaN
5885,VOLVO,V90 CROSS COUNTRY,D4 AWD,51.924€,5 puertas,Diesel,Todo terreno tracción total permanente,190 cv,Manual,6 marchas,134 gr/km,"1.176,5 km","5,1 l/100km",0,24 meses,1.969 cc,60 lts.,6,210 km/h,"8,80 s.",4250 rpm,400Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Todo Terreno,1791 kg,VOLVO V90 CROSS COUNTRY,NaN,NaN,NaN
5886,VOLVO,V90 CROSS COUNTRY,D4 AWD Aut.,54.213€,5 puertas,Diesel,Todo terreno tracción total permanente,190 cv,Automática secuencial,8 marchas,138 gr/km,"1.153,8 km","5,2 l/100km",0,24 meses,1.969 cc,60 lts.,8,210 km/h,"8,80 s.",4250 rpm,400Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Todo Terreno,1826 kg,VOLVO V90 CROSS COUNTRY,NaN,NaN,NaN


In [20]:
resultado = resultado.drop(['marca-modelo','Marca'],axis=1)

In [21]:
resultado.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso,Maletero
0,AUDI,A1,1.0 TFSI Attraction,14.426€,3 puertas,Gasolina,Tracción delantera,95 cv,Manual,5 marchas,97 gr/km,"1.071,4 km","4,2 l/100km",0,24 meses,999 cc,45 lts.,5,186 km/h,"10,90 s.",5000 rpm,160Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Berlina,1110 kg,270
1,AUDI,A1,Sportback 1.0 TFSI Adrenalin,15.296€,5 puertas,Gasolina,Tracción delantera,95 cv,Manual,5 marchas,98 gr/km,"1.046,5 km","4,3 l/100km",0,24 meses,999 cc,45 lts.,5,186 km/h,"11,10 s.",5000 rpm,160Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Berlina,1135 kg,270
2,AUDI,A1,Sportback 1.0 TFSI Attraction,15.471€,5 puertas,Gasolina,Tracción delantera,95 cv,Manual,5 marchas,97 gr/km,"1.071,4 km","4,2 l/100km",0,24 meses,999 cc,45 lts.,5,186 km/h,"11,10 s.",5000 rpm,160Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Berlina,1135 kg,270
3,AUDI,A1,Sportback 1.0 TFSI Design,15.565€,5 puertas,Gasolina,Tracción delantera,95 cv,Manual,5 marchas,97 gr/km,"1.071,4 km","4,2 l/100km",0,24 meses,999 cc,45 lts.,5,186 km/h,"11,10 s.",5000 rpm,160Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Berlina,1135 kg,270
4,AUDI,A1,1.4TDI Adrenalin,15.613€,3 puertas,Diesel,Tracción delantera,90 cv,Manual,5 marchas,95 gr/km,"1.250,0 km","3,6 l/100km",0,24 meses,1.422 cc,45 lts.,5,182 km/h,"11,40 s.",3250 rpm,230Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Berlina,1165 kg,270


In [22]:
resultado.columns

Index([u'marca', u'modelo', u'version', u'precio', u'puertas', u'combustible',
       u'traccion', u'cv', u'cambio', u'n_marchas', u'co2', u'autonomia',
       u'consumo', u'airbags', u'garantia', u'cilindrada',
       u'capacidad deposito', u'marchas', u'max_vel', u'0_a_100', u'rpm',
       u'par', u'control traccion', u'control estabilidad', u'estrellas NCAP',
       u'alarma', u'cierre centralizado', u'cierre automatico', u'tipo llave',
       u'start/stop', u'CD/mp3/dvd', u'carroceria', u'peso', u'Maletero'],
      dtype='object')

In [27]:
resultado.to_csv('datos_total.csv',encoding='utf-8')